In [1]:
from google.colab import files

# This opens a file upload dialog in Colab
uploaded = files.upload()


Saving bonuses.json to bonuses.json
Saving attendance.csv to attendance.csv
Saving employees.csv to employees.csv


#  TASK 1: Ingestion & Exploration

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("PySparkAssessment").getOrCreate()

# Read files
employees_df = spark.read.csv("employees.csv", header=True, inferSchema=True)
attendance_df = spark.read.csv("attendance.csv", header=True, inferSchema=True)
bonuses_df = spark.read.json("bonuses.json")

# Show schemas and records
employees_df.printSchema()
employees_df.show()

attendance_df.printSchema()
attendance_df.show()

bonuses_df.printSchema()
bonuses_df.show()

# Count distinct departments
print("Distinct Departments:", employees_df.select("Department").distinct().count())


root
 |-- EmpID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoinDate: date (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- ManagerID: integer (nullable = true)

+-----+------+-----------+----------+------+---------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|
+-----+------+-----------+----------+------+---------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|
|    3|Simran|Engineering|2022-07-10| 75000|        1|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|
|    5| Nisha|         HR|2023-01-05| 50000|        1|
+-----+------+-----------+----------+------+---------+

root
 |-- EmpID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)

+-----+----------+-------+
|EmpID|      Date| Status|
+-----+----------+-------+
|    1|2024-04-01|Present|
|    1|2024-04-02|Present|
|    2|2024-

# TASK 2: DataFrame Operations

In [3]:
from pyspark.sql.functions import datediff, current_date, round, col

# Tenure in years
employees_df = employees_df.withColumn("TenureYears", round(datediff(current_date(), col("JoinDate")) / 365, 1))

# Join bonuses to compute TotalCompensation
emp_bonus_df = employees_df.join(bonuses_df, "EmpID")
emp_bonus_df = emp_bonus_df.withColumn("TotalCompensation", col("Salary") + col("Bonus"))

# Filter
emp_bonus_df.filter(col("TenureYears") > 2).show()

# Employees with Manager
employees_df.filter(col("ManagerID").isNotNull()).show()


+-----+------+-----------+----------+------+---------+-----------+-----+----+---------------+-----------------+
|EmpID|  Name| Department|  JoinDate|Salary|ManagerID|TenureYears|Bonus|Year|_corrupt_record|TotalCompensation|
+-----+------+-----------+----------+------+---------+-----------+-----+----+---------------+-----------------+
|    1| Anita|         HR|2021-05-01| 55000|     NULL|        4.1| 5000|2023|           NULL|            60000|
|    2|   Raj|Engineering|2020-03-15| 80000|        1|        5.2| 7000|2023|           NULL|            87000|
|    3|Simran|Engineering|2022-07-10| 75000|        1|        2.9| 6500|2023|           NULL|            81500|
|    4| Aamir|  Marketing|2019-11-20| 60000|        1|        5.6| 6000|2023|           NULL|            66000|
|    5| Nisha|         HR|2023-01-05| 50000|        1|        2.4| 4000|2023|           NULL|            54000|
+-----+------+-----------+----------+------+---------+-----------+-----+----+---------------+-----------

# TASK 3: Aggregation

In [4]:
# Average salary by department
employees_df.groupBy("Department").avg("Salary").show()

# Employees under each manager
employees_df.groupBy("ManagerID").count().show()

# Count absences per employee
attendance_df.filter(col("Status") == "Absent")\
    .groupBy("EmpID").count().withColumnRenamed("count", "AbsenceCount").show()


+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|Engineering|    77500.0|
|         HR|    52500.0|
|  Marketing|    60000.0|
+-----------+-----------+

+---------+-----+
|ManagerID|count|
+---------+-----+
|     NULL|    1|
|        1|    4|
+---------+-----+

+-----+------------+
|EmpID|AbsenceCount|
+-----+------------+
|    4|           2|
|    2|           1|
+-----+------------+



# TASK 4: Joins


In [5]:
# Attendance percentage
attendance_summary = attendance_df.groupBy("EmpID").agg(
    count("*").alias("TotalDays"),
    count(when(col("Status") == "Present", True)).alias("PresentDays")
).withColumn("AttendancePercentage", round(col("PresentDays") / col("TotalDays") * 100, 2))

attendance_summary.show()

# Top 3 by TotalCompensation
emp_bonus_df.orderBy(col("TotalCompensation").desc()).select("EmpID", "Name", "TotalCompensation").show(3)

# Multi-level join
full_df = employees_df.join(bonuses_df, "EmpID").join(attendance_summary, "EmpID")
full_df.select("EmpID", "Name", "Department", "Bonus", "AttendancePercentage").show()


+-----+---------+-----------+--------------------+
|EmpID|TotalDays|PresentDays|AttendancePercentage|
+-----+---------+-----------+--------------------+
|    1|        2|          2|               100.0|
|    3|        2|          2|               100.0|
|    5|        2|          2|               100.0|
|    4|        2|          0|                 0.0|
|    2|        2|          1|                50.0|
+-----+---------+-----------+--------------------+

+-----+------+-----------------+
|EmpID|  Name|TotalCompensation|
+-----+------+-----------------+
|    2|   Raj|            87000|
|    3|Simran|            81500|
|    4| Aamir|            66000|
+-----+------+-----------------+
only showing top 3 rows

+-----+------+-----------+-----+--------------------+
|EmpID|  Name| Department|Bonus|AttendancePercentage|
+-----+------+-----------+-----+--------------------+
|    1| Anita|         HR| 5000|               100.0|
|    2|   Raj|Engineering| 7000|                50.0|
|    3|Simran|

# TASK 5: String & Date Functions

In [6]:
from pyspark.sql.functions import year, month, regexp_replace, lpad, concat, lit

# Year and Month from JoinDate
employees_df = employees_df.withColumn("JoinYear", year("JoinDate")).withColumn("JoinMonth", month("JoinDate"))

# Mask names
employees_df = employees_df.withColumn("MaskedName", regexp_replace("Name", r"(?<=.).", "*"))

# EmpCode as EMP001
employees_df = employees_df.withColumn("EmpCode", concat(lit("EMP"), lpad(col("EmpID").cast("string"), 3, "0")))

employees_df.select("EmpID", "Name", "MaskedName", "EmpCode", "JoinYear", "JoinMonth").show()


+-----+------+----------+-------+--------+---------+
|EmpID|  Name|MaskedName|EmpCode|JoinYear|JoinMonth|
+-----+------+----------+-------+--------+---------+
|    1| Anita|     A****| EMP001|    2021|        5|
|    2|   Raj|       R**| EMP002|    2020|        3|
|    3|Simran|    S*****| EMP003|    2022|        7|
|    4| Aamir|     A****| EMP004|    2019|       11|
|    5| Nisha|     N****| EMP005|    2023|        1|
+-----+------+----------+-------+--------+---------+



# TASK 6: Conditional & Null Handling

In [7]:
from pyspark.sql.functions import when

# Performance label
emp_bonus_df = emp_bonus_df.withColumn(
    "Performance",
    when(col("Bonus") > 6000, "High")
    .when((col("Bonus") >= 4000) & (col("Bonus") <= 6000), "Medium")
    .otherwise("Low")
)

# Handle missing ManagerID
employees_df = employees_df.fillna({"ManagerID": "No Manager"})

emp_bonus_df.select("EmpID", "Bonus", "Performance").show()
employees_df.select("EmpID", "ManagerID").show()


+-----+-----+-----------+
|EmpID|Bonus|Performance|
+-----+-----+-----------+
|    1| 5000|     Medium|
|    2| 7000|       High|
|    3| 6500|       High|
|    4| 6000|     Medium|
|    5| 4000|     Medium|
+-----+-----+-----------+

+-----+---------+
|EmpID|ManagerID|
+-----+---------+
|    1|     NULL|
|    2|        1|
|    3|        1|
|    4|        1|
|    5|        1|
+-----+---------+



# TASK 7: Spark SQL

In [8]:
# Create and use database
spark.sql("CREATE DATABASE IF NOT EXISTS hr")
spark.sql("USE hr")

# Save DataFrames as tables
employees_df.write.mode("overwrite").saveAsTable("employees")
attendance_df.write.mode("overwrite").saveAsTable("attendance")
bonuses_df.write.mode("overwrite").saveAsTable("bonuses")

# Top paid employee in each department
spark.sql("""
    SELECT Department, Name, Salary FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY Department ORDER BY Salary DESC) AS rn
        FROM employees
    ) WHERE rn = 1
""").show()

# Attendance rate by department
attendance_df.createOrReplaceTempView("attendance")
employees_df.createOrReplaceTempView("employees")

spark.sql("""
    SELECT e.Department,
           ROUND(SUM(CASE WHEN a.Status='Present' THEN 1 ELSE 0 END)*100.0 / COUNT(*), 2) AS AttendanceRate
    FROM employees e
    JOIN attendance a ON e.EmpID = a.EmpID
    GROUP BY e.Department
""").show()

# Joined after 2021 with salary > 70,000
spark.sql("""
    SELECT Name, Salary, JoinDate
    FROM employees
    WHERE JoinDate > '2021-12-31' AND Salary > 70000
""").show()


+-----------+-----+------+
| Department| Name|Salary|
+-----------+-----+------+
|Engineering|  Raj| 80000|
|         HR|Anita| 55000|
|  Marketing|Aamir| 60000|
+-----------+-----+------+

+-----------+--------------+
| Department|AttendanceRate|
+-----------+--------------+
|Engineering|         75.00|
|         HR|        100.00|
|  Marketing|          0.00|
+-----------+--------------+

+------+------+----------+
|  Name|Salary|  JoinDate|
+------+------+----------+
|Simran| 75000|2022-07-10|
+------+------+----------+



 # TASK 8: Advanced (UDF + View + Parquet)

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# UDF to classify department
def classify_dept(dept):
    return "Tech" if dept == "Engineering" else "Non-Tech"

dept_udf = udf(classify_dept, StringType())
employees_df = employees_df.withColumn("DeptCategory", dept_udf(col("Department")))
employees_df.select("Department", "DeptCategory").show()

# View and save as Parquet
emp_attendance_summary = full_df.select("EmpID", "Name", "Department", "AttendancePercentage")
emp_attendance_summary.createOrReplaceTempView("emp_attendance_summary")

emp_attendance_summary.write.mode("overwrite").partitionBy("Department").parquet("emp_attendance_summary_parquet")


+-----------+------------+
| Department|DeptCategory|
+-----------+------------+
|         HR|    Non-Tech|
|Engineering|        Tech|
|Engineering|        Tech|
|  Marketing|    Non-Tech|
|         HR|    Non-Tech|
+-----------+------------+

